Statistics (I)(2) Final Project
===

<font face="微軟正黑體">
<b>組長</b> : 劉鎮霆 B07705015 <br>
<b>組員</b> : 黃煒勛 B07705010 | 黃　心 B07705013 | 康　崴 B07705014 | 張宇承 B07705017 | 李旻叡 B07705037
</font>

## <center>Table of Contents</center>

---

[<center><font face="微軟正黑體">分析主題</font></center>](#分析主題)
[<center><font face="微軟正黑體">動機</font></center>](#動機)
[<center><font face="微軟正黑體">敘述統計 & 視覺化</font></center>](#分析主題)
[<center><font face="微軟正黑體">假設檢定</font></center>](#動機)
[<center><font face="微軟正黑體">迴歸分析</font></center>](#分析主題)
[<center><font face="微軟正黑體">結論與建議</font></center>](#動機)

---


**<font face="微軟正黑體">分析主題</font>**
---

<font face="微軟正黑體">Spotify 串流平台中歌曲特性對流行程度的影響</font>

**<font face="微軟正黑體">動機</font>**
---

<font face="微軟正黑體">
&emsp;&emsp;現今社會串流音樂蔚為主流，全世界有數億人在使用這類的服務。而在眾多的串流服務中又以 Spotify 為箇中翹楚，它有著 2.86 億個使用者，想必各個音樂發行商的競爭也是十分激烈。我們這些平時熱愛音樂的學生不禁好奇，一首歌要有怎麼樣的特性才可以在 Spotify 的使用環境中脫穎而出，得到較高的歡迎程度。若是知道歡迎程度與歌曲各項特性的關係，或許發行商可以根據此關係調整策略，獲得較高的觸及率和收益。
</font>

**<font face="微軟正黑體">資料集</font>**
---

### 1. **<font face="微軟正黑體">資料蒐集</font>**

<font face="微軟正黑體">
    &emsp;&emsp;我們首先在 kaggle 上找到
    <a href='https://www.kaggle.com/zaheenhamidani/ultimate-spotify-tracks-db'>Spotify 歌曲資料集</a>,
    其中包含了每一首歌的曲風、歌手、流行度 (popularity)、原聲性 (acousticness) 等數據。
    由於 kaggle 上的資料比較舊 (2019年3月)，再加上其中的曲風有經過人為處理 (例如將 kids 與 children 合併為一種曲風)，因此我們決定參考它蒐集資料的方法，重新蒐集歌曲資料。
</font>
<br>
<font face="微軟正黑體">
    &emsp;&emsp;抽樣時，我們採用 stratified sampling，透過 Soptipy 函式庫搜尋 Spotify 上各個曲風的歌曲 (會回傳歌曲資料的list)，再各自隨機抽樣出 100 首歌，程式碼如下。
</font>

**<font face="微軟正黑體">敘述統計 & 視覺化</font>**
---

**<font face="微軟正黑體">假設檢定</font>**
---

**<font face="微軟正黑體">迴歸分析</font>**
---

**<font face="微軟正黑體">結論與建議</font>**
---